In [4]:
pwd

'/mmfs1/data/pijarj/BC-ORG-Data/Code/Archive'

In [6]:
cd ../../../ndar_fmri/

/mmfs1/data/pijarj/ndar_fmri


In [7]:
import pandas as pd
import os
import numpy as np
from tqdm import tqdm
import shutil

In [9]:
# Pandas 
#df = pd.read_csv('image03.txt',delimiter='\t')
df = pd.read_csv('image03.txt')
df = df.iloc[1::]
df = df.iloc[~pd.isna(df['image_file']).values]
df.head(5)

ParserError: Error tokenizing data. C error: Expected 15 fields in line 209, saw 36


In [5]:
print(len(df['subjectkey'].values))
print(len(np.unique(df['subjectkey'].values)))

5509
1228


In [6]:
# Changes the paths from S3 to local cluster paths
n = len(df)
root = './image03/'
local_paths = list()
for i in tqdm(range(n)):
    s3_path = df['image_file'].values[i]
    splits = s3_path.split('/')[4::]
    relative = '/'.join(splits)
    local_path = os.path.join(root,relative)
    assert os.path.exists(os.path.join(root,relative))
    local_paths.append(local_path)

100%|██████████| 5509/5509 [00:03<00:00, 1463.50it/s]


In [7]:
print(s3_path)
print(os.path.join(root,relative))

s3://NDAR_Central_4/submission_26971/SZ0202_00_SPINECHOFIELDMAP_PA_0033.zip
./image03/SZ0202_00_SPINECHOFIELDMAP_PA_0033.zip


In [11]:
pwd

'/mmfs1/data/pijarj/ndar_fmri'

In [8]:
df['local_paths'] = local_paths

In [9]:
errors = list()
for s in tqdm(range(n)):
    subKey = df['subjectkey'].values[s]
    subFilePath = df['local_paths'].values[s]
    subFile = subFilePath.split('/')[-1]

    if not os.path.exists(os.path.join('./subjects',subKey)):
        os.mkdir(os.path.join('./subjects',subKey))
    
    try: 
        shutil.copyfile(subFilePath,os.path.join('./subjects',subKey,subFile));
    except:
        errors.append(subFilePath)

100%|██████████| 5509/5509 [00:07<00:00, 734.57it/s] 


## PART TWO

In [14]:
pwd

'/mmfs1/data/pijarj/ndar_fmri'

In [16]:
#df.to_csv('../BC-ORG-Data/Data/general_csv.csv')

In [10]:
pwd

'/mmfs1/data/pijarj/ndar_fmri'

In [122]:
indir = './subjects/'
folders = np.array(os.listdir(indir))

In [124]:
has_nii = np.array([any(['.nii' in cont for cont in os.listdir(os.path.join(indir,f))]) for f in folders])
has_tar = np.array([any(['.tar' in cont for cont in os.listdir(os.path.join(indir,f))]) for f in folders])
has_rest = np.array([any(['rest' in cont.lower() for cont in os.listdir(os.path.join(indir,f))]) for f in folders])

In [125]:
(has_rest * has_nii).sum()

708

In [161]:
exts = list()
for folder in folders:
    these_exts = [cont.split('.')[-1] for cont in os.listdir(os.path.join(indir,folder))]
    [exts.append(ext) for ext in these_exts]

In [162]:
c = pd.DataFrame(np.array(exts),columns=['key'])
c['ones'] = np.ones(len(c))
c.groupby(['key']).count()

,ones
key,
1D,1
1_1018_v1_zip,1
1_1018_v2_zip,1
BRIK,57
dcm,363
gz,2592
gz?,1
nii,436
zip,109
